# Write Questions

This notebook will demonstrate how to generate API usage questions about the Weaviate Recipes repository.

We will further illustrate how to fine-tune a Cohere Re-ranker using the synthetic dataset and then plug it into Weaviate to improve search quality.

This notebook uses a few technologies to illustrate how to achieve this:

- Generating Synthetic Queries using LangChain LCEL and Cohere Command R+
- Optimizing Query Generation with DSPy
- Fine-tuning Cohere Re-rankers
- Weaviate's `reranker-cohere` module demo

# Connect LangChain to Cohere Command R+

In [ ]:
import getpass
import os

os.environ["COHERE_API_KEY"] = "sk-foobar"

In [8]:
!pip install langchain-cohere==0.1.4

  Obtaining dependency information for langchain-cohere==0.1.4 from https://files.pythonhosted.org/packages/5b/85/96598a324a6e0b56b42634501e3d686ec64a80961f5977adbdf3d334a555/langchain_cohere-0.1.4-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-core<0.2.0,>=0.1.42 from https://files.pythonhosted.org/packages/43/8b/48b7e6de9041d2b33d5108e154b82d1bd6c47cc68f0e44cb4fcdaccf5ec7/langchain_core-0.1.52-py3-none-any.whl.metadata


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 5.1 MB/s eta 0:00:0000:01
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.4
    Uninstalling langchain-core-0.2.4:
      Successfully uninstalled langchain-core-0.2.4
  Attempting uninstall: langchain-cohere
    Found existing installation: langchain-cohere 0.1.5
    Uninstalling langchain-cohere-0.1.5:
      Successfully uninstalled langchain-cohere-0.1.5

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [9]:
from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage

chat = ChatCohere(model="command")

messages = [HumanMessage(content="1"), HumanMessage(content="2 3")]
chat.invoke(messages)

ModuleNotFoundError: No module named 'langchain_cohere'

In [ ]:
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_cohere import Chat
#model = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest", google_api_key=google_api_key)

class UpdatedPropertyValue(BaseModel):
    property_value: str

parser = PydanticOutputParser(pydantic_object=UpdatedPropertyValue)

prompt = PromptTemplate(
    template="Given the references: {references}. Update the property {property_name} by following the instruction, {instruction}. Respond with JSON with the key `property_value`.",
    input_variables=["property_name", "references", "instruction"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

prompt_and_model = prompt | model
output = prompt_and_model.invoke({
    "instruction": "Please write a compelling description for this AirBnB Listing",
    "references": "price-$300,neighborhood-Brooklyn,bedrooms=2,bathrooms=2",
    "property_name": "description"
})
parser.invoke(output)

## Update and Reflect Chain

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
chain = prompt | chat